In [ ]:
import datetime as dt
import utils
from sentinelsat import SentinelAPI
import json
import matplotlib.pyplot as plt

from rasterio import plot

In [ ]:
geojson_path = "data/var.geojson"
dates = (dt.date(2021, 8, 14), dt.date(2021, 8, 25))
credentials_path = "secrets/sentinel_api_credentials.json"

with open(credentials_path, 'r') as infile:
    credentials = json.load(infile)

api = SentinelAPI(
    credentials["username"],
    credentials["password"]
)

In [ ]:
images_df = utils.get_dataframe_between_dates(api, dates[0], dates[1], geojson_path)

In [ ]:
uuid, title = utils.select_image_cloud(images_df)
print(uuid, title)

In [ ]:
path = './data/'
utils.download_from_api(api, uuid, title, path)

In [ ]:
im_path = path + title + '.SAFE'
print(path)
img_before = utils.create_tiff_image(im_path, 'before', 'B04', 'B08', resolution=10)

In [ ]:
plt.figure(figsize=(12, 8))
plot.show(img_before)
plt.show()

In [ ]:
wildfire_date = dt.date(2021, 8, 17)
observation_interval = 14

In [ ]:
before_image, after_image = utils.get_before_after_images(
    api, dt.date(2021, 8, 17), 14, geojson_path,
    "B04", "B08", cloud_threshold=40, resolution=10
)

In [ ]:
plt.imshow(before_image)
plt.show()